# Parse README.md files, performance.csv, and add elements to a spreadsheet summarizing all of our models

In [105]:
import numpy as np
import os
import csv

In [47]:
walker = os.walk(os.path.expanduser('~/Dropbox/deep-retina/saved'), topdown=True)

In [48]:
full_paths = []
for dirs, subdirs, files in walker:
    full_paths.append([dirs, subdirs, files])

In [49]:
full_paths[0]

['/Users/lmcintosh/Dropbox/deep-retina/saved',
 ['glm',
  'lane-desktop.lane',
  'lane.local.lmcintosh',
  'lenna.nirum',
  'lenna.salamander',
  'mbp.nirum'],
 ['.DS_Store']]

In [54]:
full_paths[6][2]

['architecture.json', 'performance.csv', 'README.md']

In [53]:
'README.md' in full_paths[6][2]

True

In [119]:
all_models = []
for path in full_paths:
    readme_name = 'README.md'
    performance_name = 'performance.csv'
    if readme_name in path[2]:
        # Parse README.md
        readme_path = path[0] + '/' + readme_name
        f = open(readme_path, 'r')
        description = f.readlines()
        path_components = readme_path.split('/')
        
        # Parse performance.csv
        performance_path = path[0] + '/' + 'performance.csv'
        h = open(performance_path, 'r')
        stats = csv.reader(h)
        all_rows = []
        for row in stats:
            all_rows.append(row)
        num_epochs = len(all_rows) - 1 # -1 for the header
        
        # skip models for which we have no performance data
        if num_epochs >= 1:
            table = np.array(all_rows)
            just_numbers = table[1:, :].astype('float')

            model = {
                'type': description[0][2:-1],
                'date': description[1][3:-1],
                'machine': path_components[6],
                'folder': path_components[7],
                'stimulus': description[12][:-1],
                'experiment': description[11][:-1],
                'cells': description[14][:-1],
                'epochs': num_epochs,
                'max_train_cc': np.max(just_numbers, axis=0)[2],
                'mean_train_cc': np.mean(just_numbers, axis=0)[2],
                'max_test_cc': np.max(just_numbers, axis=0)[3],
                'mean_test_cc': np.mean(just_numbers, axis=0)[3],
            }
            all_models.append(model)
        

In [120]:
len(all_models)

55

In [129]:
all_models[50]

{'cells': 'Cell #[0, 1, 2, 3, 4]',
 'date': 'January 21, 2016',
 'epochs': 1,
 'experiment': 'Experiment 10-07-15',
 'folder': '2016-01-21 22.14.16 fixedlstm',
 'machine': 'lenna.salamander',
 'max_test_cc': -0.0207398513884,
 'max_train_cc': -0.0047959025688200001,
 'mean_test_cc': -0.0207398513884,
 'mean_train_cc': -0.0047959025688200001,
 'stimulus': 'whitenoise',
 'type': 'fixedlstm'}